# BNI.XQ

simple analysis of the open catalogue of legal deposit of italian publications ([BNI](http://bni.bncf.firenze.sbn.it/bniweb/menu.jsp))

### get the data (2015 and 2016 monographs)

download xml (unimarc xml)


    ~ curl -OJLs "http://bni.bncf.firenze.sbn.it/bniweb/scaricaxml.jsp?mese=01&anno=2015&serie=Monografie"
    ~ curl -OJLs "http://bni.bncf.firenze.sbn.it/bniweb/scaricaxml.jsp?mese=02&anno=2015&serie=Monografie"
    ~ curl -OJLs "http://bni.bncf.firenze.sbn.it/bniweb/scaricaxml.jsp?mese=03&anno=2015&serie=Monografie"
    ~ curl -OJLs "http://bni.bncf.firenze.sbn.it/bniweb/scaricaxml.jsp?mese=01&anno=2016&serie=Monografie"

    ~ sha1sum Monografie*.xml
    7c226c88daefd7b145ebb0bc01d621ba9f3ea9b3  Monografie201501.xml
    204134fef0f5275f466feb9c6a018c794fadd07b  Monografie201502.xml
    bdbcab246290b9d2e0db3b7279bd32ea20ea6ef3  Monografie201503.xml
    c8e56442bc5c8a1e7fb9e31731108ba586993c17  Monografie201601.xml
    
### install basex http://basex.org

deb      `~ apt-get install basex`  
macos    `~ brew install basex`


### create the database

    ~ basex -c "create database bni"
    ~ basex -i bni -c "add Monografie201501.xml; add Monografie201502.xml;\
                       add Monografie201503.xml; Monografie201601.xml"

### start the database
    ~ basexserver
    
### install jupyter and basexclient.py

    ~ pip3 install jupyter
    ~ pip3 install basexclient


In [2]:
from BaseXClient import BaseXClient

In [3]:
session = BaseXClient.Session('127.0.0.1', 1984, 'admin', 'admin')

paths to extract and count. here a quick (but uncomplete) [reference](http://unimarc-it.wikidot.com/) of unimarc fields. here the [unimarc xml schema](http://www.bncf.firenze.sbn.it/progetti/unimarc/slim/documentation/unimarcslim.html)

In [4]:
publisher = '//rec/df[@t="210"]/sf[@c="c"]'
publisher_city = '//rec/df[@t="210"]/sf[@c="a"]'
subject = '//rec/df[@t="606"]/sf[@c="a"]'

In [9]:
def count(path, limit):
    q = '''let $db := db:open("bni")
    let $result :=
        for $publisher in distinct-values($db{0})
              let $count :=  count(index-of($db{0}, $publisher))
              order by $count descending
        return concat($publisher, ", ", $count)
    for $limited at $lim in subsequence($result, 1, {1})          
        return $limited'''.format(path, limit)
    query = session.query(q)
    for _, item in query.iter():
        print(item)

### top 10 publishers

In [10]:
count(publisher, 10)

Angeli, 324
Giuffrè, 213
, 188
Il mulino, 149
Mondadori, 113
Skira, 110
Einaudi, 104
Carocci, 104
Rubbettino, 100
Marsilio, 84


### top 10 location of publishers

In [11]:
count(publisher_city, 10)

Milano, 1482
Roma, 690
Torino, 375
Bologna, 264
Firenze, 262
Napoli, 249
[Milano], 168
Pisa, 129
Venezia, 127
Soveria Mannelli, 103


### top 30 subjects

In [12]:
count(subject, 30)

Arte, 46
Italia, 40
Politica, 38
Aziende, 37
Cultura, 26
Economia, 25
Letteratura, 25
Psicanalisi, 24
Letteratura italiana, 24
Architettura, 22
Diritto, 21
Donne, 21
Diritto civile, 19
Educazione, 19
Bambini, 18
Sviluppo economico, 18
Diritto commerciale, 17
Lingua italiana, 17
Guerra mondiale 1914-1918, 16
Ebrei, 15
Città, 14
Pittura, 14
Immigrati, 13
Società, 13
Filosofia, 12
Famiglie, 11
Psicoterapia, 11
Arte italiana, 11
Paesaggio, 11
Condizioni socioeconomiche, 10
